In [1]:
import torch
from torchsummary import summary
from src.utils import load_data
from src.transformations import *
from src.models.pretrained_models import ResNetPretrained
from src.model_trainer import ModelTrainer
import itertools

In [2]:
device_str = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_str)
device

device(type='cpu')

In [3]:
train_loader = load_data(
    "./data/train",
    batch_size=256,
    shuffle=True,
    transform=resize_transform(),
    num_workers=2,
)

In [4]:
resnet = ResNetPretrained()

In [ ]:
n_epochs = 1  # set to 10 ?
max_batches = 1  # set to None

In [8]:
training_process_grid = {
    "optimizer_type": ["adam", "sgd"],
    "learning_rate": [0.01, 0.001, 0.0001],
}

regularization_grid = {
    "weight_decay": [0.01, 0.001, 0.0001],
    "dropout": [0.2, 0.5, 0.7],
}

# grid search for training parameters with set L1 regularizer = 0 & dropout = 0
# grid search for regularization parameters with set optimizer = adam & learning_rate = 0.001

training_param_combinations = list(itertools.product(*training_process_grid.values()))
regularization_param_combinations = list(
    itertools.product(*regularization_grid.values())
)

In [13]:
for optimizer_type, learning_rate in training_param_combinations:
    trainer = ModelTrainer(
        model=resnet,
        train_loader=train_loader,
        device=device,
        weight_decay=0,
        optimizer_type=optimizer_type,
        dropout=0,
        learning_rate=learning_rate,
        log_file=f"resnet_{optimizer_type}_{learning_rate}.json",
        save_dir="./saved_models/resnet/training_params",
        max_batches=max_batches,
    )
    trainer.train(n_epochs)

is cuda False
Batch count 0
Epoch 1/1, Loss: 1.2602, Accuracy: 0.5352, F1-score: 0.5383
Model saved to ./saved_models/resnet/training_params\ResNetPretrained_1.pth
Training log saved to ./saved_models/resnet/training_params\resnet_adam_0.01.json
is cuda False
Batch count 0
Epoch 1/1, Loss: 7.5715, Accuracy: 0.2383, F1-score: 0.1562
Model saved to ./saved_models/resnet/training_params\ResNetPretrained_1.pth
Training log saved to ./saved_models/resnet/training_params\resnet_adam_0.001.json
is cuda False
Batch count 0
Epoch 1/1, Loss: 5.5652, Accuracy: 0.3047, F1-score: 0.2116
Model saved to ./saved_models/resnet/training_params\ResNetPretrained_1.pth
Training log saved to ./saved_models/resnet/training_params\resnet_adam_0.0001.json
is cuda False
Batch count 0
Epoch 1/1, Loss: 5.4493, Accuracy: 0.3008, F1-score: 0.2138
Model saved to ./saved_models/resnet/training_params\ResNetPretrained_1.pth
Training log saved to ./saved_models/resnet/training_params\resnet_sgd_0.01.json
is cuda False


In [14]:
for weight_decay, dropout in regularization_param_combinations:
    trainer = ModelTrainer(
        model=resnet,
        train_loader=train_loader,
        device=device,
        optimizer_type="adam",
        learning_rate=0.001,
        weight_decay=weight_decay,
        dropout=dropout,
        log_file=f"resnet_{weight_decay}_{dropout}.json",
        save_dir="./saved_models/resnet/regularization_params",
        max_batches=max_batches,
    )
    trainer.train(n_epochs)

is cuda False
Batch count 0
Epoch 1/1, Loss: 5.0680, Accuracy: 0.3203, F1-score: 0.2255
Model saved to ./saved_models/resnet/regularization_params\ResNetPretrained_1.pth
Training log saved to ./saved_models/resnet/regularization_params\resnet_0.01_0.2.json
is cuda False
Batch count 0
Epoch 1/1, Loss: 3.8199, Accuracy: 0.3398, F1-score: 0.2342
Model saved to ./saved_models/resnet/regularization_params\ResNetPretrained_1.pth
Training log saved to ./saved_models/resnet/regularization_params\resnet_0.01_0.5.json
is cuda False
Batch count 0
Epoch 1/1, Loss: 2.7944, Accuracy: 0.3984, F1-score: 0.2638
Model saved to ./saved_models/resnet/regularization_params\ResNetPretrained_1.pth
Training log saved to ./saved_models/resnet/regularization_params\resnet_0.01_0.7.json
is cuda False
Batch count 0
Epoch 1/1, Loss: 2.2295, Accuracy: 0.4062, F1-score: 0.2750
Model saved to ./saved_models/resnet/regularization_params\ResNetPretrained_1.pth
Training log saved to ./saved_models/resnet/regularization_

### Best model to be tested on dataset altered through data augmentation techniques

In [15]:
train_rotated = load_data(
    "./data/train",
    batch_size=256,
    shuffle=True,
    transform=random_rotation_transform(),
    num_workers=2,
)